In [1]:
import pandas as pd
loans = pd.read_csv('LoanStats3a.csv', skiprows=1, low_memory=False)
half_count = len(loans) / 2
loans = loans.dropna(thresh=half_count, axis=1)
loans = loans.drop(['desc'],axis=1)
loans.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens,hardship_flag,disbursement_method,debt_settlement_flag
0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,...,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,Cash,N
1,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,...,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,Cash,N
2,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,...,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,Cash,N
3,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,...,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,Cash,N
4,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,...,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,Cash,N


 Let's pay attention to any features that:
* leak information from the future (after the loan has already been funded)
* don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
* formatted poorly and need to be cleaned up
* require more data or a lot of processing to turn into a useful feature
* contain redundant information
_______
`funded_amnt`: leaks data from the future (after the loan is already started to be funded)  
`funded_amnt_inv`: also leaks data from the future (after the loan is already started to be funded)  
`grade`: contains redundant information as the interest rate column (int_rate)  
`sub_grade`: also contains redundant information as the interest rate column (int_rate)  
`emp_title`: requires other data and a lot of processing to potentially be useful  
`issue_d`: leaks data from the future (after the loan is already completed funded)  
`grade` and `sub_grade`: values are categorical, the `int_rate` column contains continuous values, which are better suited for machine learning. 

In [2]:
loans = loans.drop(["funded_amnt", "funded_amnt_inv", "emp_title", "issue_d","grade", "sub_grade"], axis=1)

`zip_code`: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)  
`out_prncp`: leaks data from the future, (after the loan already started to be paid off)  
`out_prncp_inv`: also leaks data from the future, (after the loan already started to be paid off)   
`total_pymnt`: also leaks data from the future, (after the loan already started to be paid off)  
`total_pymnt_inv`: also leaks data from the future, (after the loan already started to be paid off)  
`total_rec_prncp`: also leaks data from the future, (after the loan already started to be paid off)  
`out_prncp` and `out_prncp_inv` : both describe the outstanding principal amount for a loan, which is the remaining amount the borrower still owes. Properties of the loan after it's fully funded.Information not available to an investor before the loan is fully funded and we don't want to include it in our model.   
`total_pymnt` : describe propertie of the loan after it's fully funded. Information not available to an investor before the loan is fully funded and we don't want to include it in our model.

In [3]:
loans = loans.drop(["zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp"], axis=1)

`total_rec_int`: leaks data from the future, (after the loan already started to be paid off),  
`total_rec_late_fee`: also leaks data from the future, (after the loan already started to be paid off),  
`recoveries`: also leaks data from the future, (after the loan already started to be paid off),  
`collection_recovery_fee`: also leaks data from the future, (after the loan already started to be paid off),  
`last_pymnt_d`: also leaks data from the future, (after the loan already started to be paid off),   
`last_pymnt_amnt`: also leaks data from the future, (after the loan already started to be paid off),   
`debt_settlement_flag`: also leaks data from the future, (after the loan already started to be paid off).   
All of these columns leak data from the future, meaning that they're describing aspects of the loan after it's already been fully funded and started to be paid off by the borrower.

In [4]:
loans = loans.drop(["total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt", "debt_settlement_flag"], axis=1)
print(loans.iloc[0])
print(loans.shape[1])

loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                 Jan-1985
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                          83.7%
total_acc                               9
initial_list_status                     f
last_credit_pull_d               A

### Target definition

We should use the `loan_status` column, since it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower.   

Currently, this column contains text values and we need to convert it to a numerical one for training a model. Let's explore the different values in this column and come up with a strategy for converting the values in this column.

In [5]:
print(loans['loan_status'].value_counts())

Fully Paid                                             34116
Charged Off                                             5670
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Name: loan_status, dtype: int64


 | Loan Status                                        |	Count	 | Meaning  |
 |:---------------------------------------------------| :--------| :--------|   
 | Fully Paid                                         |	33136	 | Loan has been fully paid off.  |
 | Charged Off                                        |	5634	 | Loan for which there is no longer a reasonable expectation of further payments.  |
 | Does not meet the credit policy. Status:Fully Paid |	1988	 | While the loan was paid off, the loan application   today would no longer meet the credit policy and wouldn't be approved on to the marketplace.  |
 | Does not meet the credit policy. Status:Charged Off| 761	 | While the loan was charged off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.  |

___

In Grace Period	20	The loan is past due but still in the grace period of 15 days.  
Late (16-30 days)	8	Loan hasn't been paid in 16 to 30 days (late on the current payment).  
Late (31-120 days)	24	Loan hasn't been paid in 31 to 120 days (late on the current payment).  
Current	961	Loan is up to date on current payments.  
Default	3	Loan is defaulted on and no payment has been made for more than 121 days.  

From the investor's perspective, we're interested in trying to predict which loans will be paid off on time and which ones won't be. Only the Fully Paid and Charged Off values describe the final outcome of the loan. The other values describe loans that are still on going and where the jury is still out on if the borrower will pay back the loan on time or not. While the Default status resembles the Charged Off status, in Lending Club's eyes, loans that are charged off have essentially no chance of being repaid while default ones have a small chance. You can read about the difference here.

Since we're interested in being able to predict which of these 2 values a loan will fall under, we can treat the problem as a binary classification one. Let's remove all the loans that don't contain either Fully Paid and Charged Off as the loan's status and then transform the Fully Paid values to 1 for the positive case and the Charged Off values to 0 for the negative case. While there are a few different ways to transform all of the values in a column, we'll use the Dataframe method replace. According to the documentation, we can pass the replace method a nested mapping dictionary in the following format:

Lastly, one thing we need to keep in mind is the class imbalance between the positive and negative cases. While there are 33,136 loans that have been fully paid off, there are only 5,634 that were charged off. This class imbalance is a common problem in binary classification and during training, the model ends up having a strong bias towards predicting the class with more observations in the training set and will rarely predict the class with less observations. The stronger the imbalance, the more biased the model becomes. There are a few different ways to tackle this class imbalance, which we'll explore later.

In [6]:
loans = loans[(loans['loan_status'] == "Fully Paid") | (loans['loan_status'] == "Charged Off")]

status_replace = {
    "loan_status" : {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}
loans = loans.replace(status_replace)

We'll need to compute the number of unique values in each column and drop the columns that contain only one unique value. While the Series method unique returns the unique values in a column, it also counts the Pandas missing value object nan as a value:

In [7]:
orig_columns = loans.columns
drop_columns = []
for col in orig_columns:
    col_series = loans[col].dropna().unique()
    if len(col_series) == 1:
        drop_columns.append(col)
loans = loans.drop(drop_columns, axis=1)
print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens', 'hardship_flag', 'disbursement_method']


In [8]:
null_counts = loans.isnull().sum()
print(null_counts)

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1078
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64


While most of the columns have 0 missing values, 2 columns have 50 or less rows with missing values, and 1 column, pub_rec_bankruptcies, contains 697 rows with missing values. Let's remove columns entirely where more than 1% of the rows for that column contain a null value. In addition, we'll remove the remaining rows containing null values.

This means that we'll keep the following columns and just remove rows containing missing values for them:

title
revol_util
last_credit_pull_d
and drop the pub_rec_bankruptcies column entirely since more than 1% of the rows have a missing value for this column.

In [9]:
loans = loans.drop("pub_rec_bankruptcies", axis=1)
loans = loans.dropna(axis=0)
print(loans.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


While the numerical columns can be used natively with scikit-learn, the object columns that contain text need to be converted to numerical data types. Let's return a new Dataframe containing just the object columns so we can explore them in more depth

In [10]:
object_columns_df = loans.select_dtypes(include=["object"])
print(object_columns_df.iloc[0])

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line          Jan-1985
revol_util                   83.7%
last_credit_pull_d        Apr-2018
Name: 0, dtype: object


Some of the columns seem like they represent categorical values, but we should confirm by checking the number of unique values in those columns:

home_ownership: home ownership status, can only be 1 of 4 categorical values according to the data dictionary,
verification_status: indicates if income was verified by Lending Club,
emp_length: number of years the borrower was employed upon time of application,
term: number of payments on the loan, either 36 or 60,
addr_state: borrower's state of residence,
purpose: a category provided by the borrower for the loan request,
title: loan title provided the borrower,
There are also some columns that represent numeric values, that need to be converted:

int_rate: interest rate of the loan in %,
revol_util: revolving line utilization rate or the amount of credit the borrower is using relative to all available credit

Based on the first row's values for purpose and title, it seems like these columns could reflect the same information. Let's explore the unique value counts separately to confirm if this is true.

Lastly, some of the columns contain date values that would require a good amount of feature engineering for them to be potentially useful:

earliest_cr_line: The month the borrower's earliest reported credit line was opened,
last_credit_pull_d: The most recent month Lending Club pulled credit for this loan.
Since these date features require some feature engineering for modeling purposes, let's remove these date columns from the Dataframe.

In [11]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for c in cols:
    print(c, "\n", loans[c].value_counts(), "\n")

home_ownership 
 RENT        18471
MORTGAGE    17242
OWN          2837
OTHER          96
NONE            3
Name: home_ownership, dtype: int64 

verification_status 
 Not Verified       16468
Verified           12377
Source Verified     9804
Name: verification_status, dtype: int64 

emp_length 
 10+ years    8897
< 1 year     4576
2 years      4389
3 years      4094
4 years      3435
5 years      3279
1 year       3240
6 years      2227
7 years      1771
8 years      1483
9 years      1258
Name: emp_length, dtype: int64 

term 
  36 months    28234
 60 months    10415
Name: term, dtype: int64 

addr_state 
 CA    6907
NY    3711
FL    2779
TX    2674
NJ    1825
IL    1487
PA    1481
VA    1378
GA    1358
MA    1313
OH    1190
MD    1034
AZ     832
WA     807
CO     769
NC     761
CT     734
MI     688
MO     661
MN     591
NV     482
SC     464
WI     445
OR     436
AL     433
LA     426
KY     323
OK     293
KS     260
UT     253
AR     235
DC     212
RI     197
NM     184
HI     169
W

The home_ownership, verification_status, emp_length, term, and addr_state columns all contain multiple discrete values. We should clean the emp_length column and treat it as a numerical one since the values have ordering (2 years of employment is less than 8 years).

First, let's look at the unique value counts for the purpose and title columns to understand which column we want to keep.

In [12]:
print(loans["purpose"].value_counts(), "\n")
print(loans["title"].value_counts(), "\n")

debt_consolidation    18262
credit_card            5004
other                  3824
home_improvement       2884
major_purchase         2109
small_business         1783
car                    1497
wedding                 934
medical                 668
moving                  557
house                   369
vacation                351
educational             312
renewable_energy         95
Name: purpose, dtype: int64 

Debt Consolidation                                                                  2149
Debt Consolidation Loan                                                             1695
Personal Loan                                                                        643
Consolidation                                                                        510
debt consolidation                                                                   489
Credit Card Consolidation                                                            349
Home Improvement                            

The home_ownership, verification_status, emp_length, and term columns each contain a few discrete categorical values. We should encode these columns as dummy variables and keep them.

It seems like the purpose and title columns do contain overlapping information but we'll keep the purpose column since it contains a few discrete values. In addition, the title column has data quality issues since many of the values are repeated with slight modifications (e.g. Debt Consolidation and Debt Consolidation Loan and debt consolidation).

We erred on the side of being conservative with the 10+ years, < 1 year and n/a mappings. We assume that people who may have been working more than 10 years have only really worked for 10 years. We also assume that people who've worked less than a year or if the information is not available that they've worked for 0. This is a general heuristic but it's not perfect.

Lastly, the addr_state column contains many discrete values and we'd need to add 49 dummy variable columns to use it for classification. This would make our Dataframe much larger and could slow down how quickly the code runs. Let's remove this column from consideration.

In [13]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
loans = loans.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1)
loans["int_rate"] = loans["int_rate"].str.rstrip("%").astype("float")
loans["revol_util"] = loans["revol_util"].str.rstrip("%").astype("float")
loans = loans.replace(mapping_dict)

Let's now encode the home_ownership, verification_status, purpose, and term columns as dummy variables so we can use them in our model.

In [14]:
cat_columns = ["home_ownership", "verification_status", "purpose", "term"]
dummy_df = pd.get_dummies(loans[cat_columns])
loans = pd.concat([loans, dummy_df], axis=1)
loans = loans.drop(cat_columns, axis=1)

In [15]:
loans.head()

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,5000.0,10.65,162.87,10,24000.0,1,27.65,0.0,1.0,3.0,...,0,0,0,0,0,0,0,0,1,0
1,2500.0,15.27,59.83,0,30000.0,0,1.00,0.0,5.0,3.0,...,0,0,0,0,0,0,0,0,0,1
2,2400.0,15.96,84.33,10,12252.0,1,8.72,0.0,2.0,2.0,...,0,0,0,0,0,1,0,0,1,0
3,10000.0,13.49,339.31,10,49200.0,1,20.00,0.0,1.0,10.0,...,0,0,0,1,0,0,0,0,1,0
4,3000.0,12.69,67.79,1,80000.0,1,17.94,0.0,0.0,15.0,...,0,0,0,1,0,0,0,0,0,1


In [16]:
loans.to_csv("cleaned_loans_2007.csv")